# Непрерывные счётчики времени

## TL;DR

Best practice: если требуется монотонный счётчик с высокой точностью, можно использовать `default_timer` из модуля `timeit`.

## Определения времён

*Wall time* - разница между временами конца и начала выполнения программы.

*CPU time (process time)* - время, в течение которого CPU выполнял программу.

*User time* - время, в течение которого CPU выполнял программу в user space.

*System time* - время, в течение которого CPU выполнял программу в kernel space.

## `time`

In [97]:
import time

time.sleep(1)

Задерживает исполнение вызывающего треда на переданное число секунд. Если получен сигнал и обработчик сигнала бросает исключение задержка может длится меньше указанного времени.

Обработчик `SIGINT` бросает `KeyboardInterrupt`.

In [98]:
import signal, threading

main_thread_ident = threading.current_thread().ident

def wait_and_signal():
  time.sleep(5)
  signal.pthread_kill(main_thread_ident, signal.SIGINT)

threading.Thread(target=wait_and_signal).start()

time_0 = time.time()
try:
  time.sleep(10)
except KeyboardInterrupt:
  pass
time_1 = time.time()

print('Должен был спать 10 секунд, спал %d секунд.' % (time_1 - time_0))

Должен был спать 10 секунд, спал 5 секунд.


Важной характеристикой счётчиков является точность. Функция `clock_getres(clk_id)` возвращает точность счётчика с переданным `clk_id`.

In [99]:
import pandas as pd

rows = []
possible_clk_id_constants = ['CLOCK_BOOTTIME', 'CLOCK_HIGHRES', 'CLOCK_MONOTONIC', 'CLOCK_MONOTONIC_RAW', 'CLOCK_PROCESS_CPUTIME_ID', 'CLOCK_PROF', \
                    'CLOCK_THREAD_CPUTIME_ID', 'CLOCK_UPTIME', 'CLOCK_REALTIME']
for clk_id_constant in possible_clk_id_constants:
  if hasattr(time, clk_id_constant):
    rows.append([clk_id_constant, time.clock_getres(getattr(time, clk_id_constant))])
df = pd.DataFrame(data=rows, columns=['clk_id', 'resolution'])
blankIndex=[''] * len(df)
df.index=blankIndex
df

,clk_id,resolution
,CLOCK_MONOTONIC,1.000000e-06
,CLOCK_MONOTONIC_RAW,1.000000e-09
,CLOCK_PROCESS_CPUTIME_ID,1.000000e-06
,CLOCK_THREAD_CPUTIME_ID,1.000000e-09
,CLOCK_REALTIME,1.000000e-06


```
time_0 = timer()
# code
time_1 = timer()
```
Счётчики, для которых гарантируется, что `time_0` $\le$`time_1`, называются монотонными. Для остальных счётчиков `time_0` может быть больше чем `time_1`, если между вызовами (# code) происходит сдвиг системных часов. Системные часы могут быть сдвинуты вследствие действий системного администратора или NTP adjustments.

In [100]:
time_0 = time.time()
time.clock_settime(time.CLOCK_REALTIME, time_0 - 1)
time_1 = time.time()
time_1 - time_0

-0.9914281368255615

In [101]:
clock_names = ['monotonic', 'perf_counter', 'process_time', 'thread_time', 'time']
rows = [None] * len(clock_names)
for i, name in enumerate(clock_names):
  rows[i] = {'clock name':name}
  rows[i].update(time.get_clock_info(name).__dict__)
df = pd.DataFrame(data=rows, columns=rows[0].keys())
blankIndex=[''] * len(df)
df.index=blankIndex
df

,clock name,implementation,monotonic,adjustable,resolution
,monotonic,mach_absolute_time(),True,False,1.000000e-09
,perf_counter,mach_absolute_time(),True,False,1.000000e-09
,process_time,clock_gettime(CLOCK_PROCESS_CPUTIME_ID),True,False,1.000000e-06
,thread_time,clock_gettime(CLOCK_THREAD_CPUTIME_ID),True,False,1.000000e-09
,time,clock_gettime(CLOCK_REALTIME),False,True,1.000000e-06


In [ ]:
time.clock_gettime(clk_id) → float
time.clock_gettime_ns(clk_id) → int
time.clock_settime(clk_id, time: float)
time.clock_settime_ns(clk_id, time: int)

In [103]:
time_0 = time.monotonic()
time.sleep(1)
time_1 = time.monotonic()
time_1 - time_0

1.001488822000283

Возвращает монотонное *wall time* время в секундах с неопределённого момента. Значение не зависит от изменений системных часов.

In [104]:
time_0 = time.perf_counter()
time.sleep(1)
time_1 = time.perf_counter()
time_1 - time_0

1.000951587000145

Возвращает *wall time* время счётчика с наибольшей точностью в секундах с неопределённого момента.

In [105]:
time_0 = time.process_time()
time.sleep(1)
time_1 = time.process_time()
time_1 - time_0

0.0034200000000002007

Возвращает сумму *system time* и *user time* текущего процесса в секундах с неопределённого момента.

In [106]:
time_0 = time.thread_time()
time.sleep(1)
time_1 = time.thread_time()
time_1 - time_0

0.0002306960000000302

Аналог `time.process_time()` для тредов.

In [107]:
time.time()

1553727504.2132218

Возвращает *wall time* с эпохи в секундах. Обычно эпоха = 01.01.1970 00:00:00 (UTC), также эпоху возвращает `gmtime(0)`.

Если между вызовами системные часы были изменены, то возвращаемые значения могут быть немонотонными.

In [108]:
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

Есть аналогичные функции, возвращающие время в наносекундах, тип `int`: `time.monotonic_ns()`, `time.perf_counter_ns()`, `time.process_time_ns()`, `time.thread_time_ns()`,  `time.time_ns()`.

In [ ]:
time.pthread_getcpuclockid(thread_id) # возвращает clk_id счётчика, который возвращает CPU time для thread c переданным thread_id

## `asyncio.loop.time()`

In [110]:
import inspect, asyncio

lines = inspect.getsource(asyncio.BaseEventLoop.time)
print(lines)

    def time(self):
        """Return the time according to the event loop's clock.

        This is a float expressed in seconds since an epoch, but the
        epoch, precision, accuracy and drift are unspecified and may
        differ per event loop.
        """
        return time.monotonic()



In [111]:
import asyncio

loop = asyncio.new_event_loop()

for i in range(5):
  await asyncio.sleep(1)
  print(loop.time())

loop.stop()
loop.close()

3092.072869279
3093.078410952
3094.082194017
3095.086349509
3096.091268329


## `timeit`

Модуль применяется для бенчмаркинга.

Пример

In [112]:
import timeit
timeit.timeit(stmt='"g" in text', setup='text = "sample string"')

0.055205082000156835

Основные функции

In [ ]:
default_timer = time.perf_counter

timeit(stmt='pass', setup='pass', timer=default_timer, number=1000000, globals=None) 
# вызывает setup 1 раз, вызывает stmt number раз, возвращает среднее время
# перед замером времени выключается сборщик мусора

repeat(stmt='pass', setup='pass', timer=default_timer, repeat=5, number=1000000, globals=None) 
# вызывает timeit repeat раз, возвращает лист времён

T = Timer(stmt="pass", setup="pass", timer=default_timer, globals=None)
T.autorange(callback=None)
# вызывает timeit с возрастающими значениями number пока замеряемое время < 0.2 секунд

Вычисление времени исполнения производится с помощью счётчика `timer`, значение по умолчанию - `perf_counter` из модуля `time`.